In [1]:
import pandas as pd

In [2]:
df_1 = pd.read_csv('/Users/ericvincent/code/dynamic_portfolio/raw_data/oil_return.csv')

In [3]:
df_1 = df_1.drop(columns='Unnamed: 0')

In [4]:
df_1.to_csv('oil.csv')

In [5]:
df = pd.read_csv('/Users/ericvincent/code/dynamic_portfolio/raw_data/treasury_yield.csv')

In [6]:
df.head()

,Unnamed: 0,timestamp,value_10years,value_2years,value
0,0,2022-11-23,3.71,4.46,-0.75
1,1,2022-11-22,3.76,4.47,-0.71
2,2,2022-11-21,3.83,4.48,-0.65
3,3,2022-11-18,3.82,4.51,-0.69
4,4,2022-11-17,3.77,4.43,-0.66


In [7]:
df = df.rename(columns = {'value':'10_2_spread'})

In [8]:
df['10_2_spread'] = df['10_2_spread']/100

In [9]:
df

,Unnamed: 0,timestamp,value_10years,value_2years,10_2_spread
0,0,2022-11-23,3.71,4.46,-0.0075
1,1,2022-11-22,3.76,4.47,-0.0071
2,2,2022-11-21,3.83,4.48,-0.0065
3,3,2022-11-18,3.82,4.51,-0.0069
4,4,2022-11-17,3.77,4.43,-0.0066
...,...,...,...,...,...
5968,5968,2000-01-07,6.52,6.31,0.0021
5969,5969,2000-01-06,6.57,6.35,0.0022
5970,5970,2000-01-05,6.62,6.38,0.0024
5971,5971,2000-01-04,6.49,6.30,0.0019


In [10]:
df = df.sort_values('date', ascending=True)

KeyError: 'date'

In [ ]:
df = df.drop(columns='Unnamed: 0')

In [ ]:
df

In [ ]:
df.to_csv('us_yields.csv')

In [ ]:
yields = pd.read_csv('/Users/ericvincent/code/dynamic_portfolio/notebooks/us_yields.csv')

In [ ]:
yields

In [11]:
chomage = pd.read_csv('/Users/ericvincent/code/dynamic_portfolio/raw_data/economic_indicatorUNEMPLOYMENT.csv')

In [12]:
chomage

,timestamp,value
0,2022-10-01,3.7
1,2022-09-01,3.5
2,2022-08-01,3.7
3,2022-07-01,3.5
4,2022-06-01,3.6
...,...,...
893,1948-05-01,3.5
894,1948-04-01,3.9
895,1948-03-01,4.0
896,1948-02-01,3.8


In [ ]:
chomage.dtypes

In [13]:
chomage['timestamp'] = pd.to_datetime(chomage['timestamp'])

In [14]:
chomage.dtypes

timestamp    datetime64[ns]
value               float64
dtype: object

In [18]:
chomage = chomage.rename(columns={'timestamp':'date'})

In [16]:
chomage['unemployment_rate'] = chomage['unemployment_rate']/100

In [20]:
chomage

,date,unemployment_rate
0,2022-10-01,0.037
1,2022-09-01,0.035
2,2022-08-01,0.037
3,2022-07-01,0.035
4,2022-06-01,0.036
...,...,...
893,1948-05-01,0.035
894,1948-04-01,0.039
895,1948-03-01,0.040
896,1948-02-01,0.038


In [21]:
chomage_mask = chomage['date']>'1999-12-01'

In [29]:
chomage = chomage[chomage_mask]

In [30]:
chomage

,date,unemployment_rate
0,2022-10-01,0.037
1,2022-09-01,0.035
2,2022-08-01,0.037
3,2022-07-01,0.035
4,2022-06-01,0.036
...,...,...
269,2000-05-01,0.040
270,2000-04-01,0.038
271,2000-03-01,0.040
272,2000-02-01,0.041


In [31]:
chomage.to_csv('unemployment.csv')

In [32]:
payroll = pd.read_csv('/Users/ericvincent/code/dynamic_portfolio/raw_data/economic_indicatorNONFARM_PAYROLL.csv')

In [41]:
payroll = payroll.rename(columns={'timestamp':'date'})

In [39]:
payroll['timestamp'] = pd.to_datetime(payroll['timestamp'])

In [42]:
payroll

,date,non_farm_payroll
0,2022-10-01,154369
1,2022-09-01,153197
2,2022-08-01,152674
3,2022-07-01,152258
4,2022-06-01,152607
...,...,...
1001,1939-05-01,30145
1002,1939-04-01,29786
1003,1939-03-01,29804
1004,1939-02-01,29394


In [43]:
payroll_mask = payroll['date']>'1999-12-01'

In [44]:
payroll = payroll[payroll_mask]

In [46]:
payroll['payroll_change'] = payroll['non_farm_payroll'].pct_change()

/var/folders/kt/jxv1lsvj67x2t9_8lqfh68tw0000gn/T/ipykernel_42974/1008215945.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  payroll['payroll_change'] = payroll['non_farm_payroll'].pct_change()


In [47]:
payroll

,date,non_farm_payroll,payroll_change
0,2022-10-01,154369,NaN
1,2022-09-01,153197,-0.007592
2,2022-08-01,152674,-0.003414
3,2022-07-01,152258,-0.002725
4,2022-06-01,152607,0.002292
...,...,...,...
269,2000-05-01,132705,-0.003843
270,2000-04-01,131749,-0.007204
271,2000-03-01,130751,-0.007575
272,2000-02-01,129655,-0.008382


In [48]:
payroll.to_csv('nonfamr_payroll.csv')

In [49]:
temp = pd.read_csv('/Users/ericvincent/code/dynamic_portfolio/raw_data/economic_indicatorINFLATION_EXPECTATION.csv')

In [56]:
temp = temp.rename(columns = {'timestamp':'date'})

In [53]:
temp['inf_exp'] = temp['inf_exp'] /100

In [54]:
temp

,timestamp,inf_exp
0,2022-10-01,0.050
1,2022-09-01,0.047
2,2022-08-01,0.048
3,2022-07-01,0.052
4,2022-06-01,0.053
...,...,...
533,1978-05-01,0.069
534,1978-04-01,0.067
535,1978-03-01,0.063
536,1978-02-01,0.064


In [55]:
temp['timestamp'] = pd.to_datetime(temp['timestamp'])

In [57]:
temp

,date,inf_exp
0,2022-10-01,0.050
1,2022-09-01,0.047
2,2022-08-01,0.048
3,2022-07-01,0.052
4,2022-06-01,0.053
...,...,...
533,1978-05-01,0.069
534,1978-04-01,0.067
535,1978-03-01,0.063
536,1978-02-01,0.064


In [58]:
temp_mask = temp['date'] >= '2000-01-01'

In [60]:
temp = temp[temp_mask]

In [61]:
temp

,date,inf_exp
0,2022-10-01,0.050
1,2022-09-01,0.047
2,2022-08-01,0.048
3,2022-07-01,0.052
4,2022-06-01,0.053
...,...,...
269,2000-05-01,0.030
270,2000-04-01,0.032
271,2000-03-01,0.032
272,2000-02-01,0.029


In [62]:
temp.to_csv('inflation_expectation.csv')

In [174]:
url = '/Users/ericvincent/code/dynamic_portfolio/raw_data/treasury_yields.csv'
temp = pd.read_csv(url, index_col=0)

In [175]:
temp

,date,value_10years,value_2years,10Y-2Y spread
0,2022-11-23,0.0371,0.0446,-0.0075
1,2022-11-22,0.0376,0.0447,-0.0071
2,2022-11-21,0.0383,0.0448,-0.0065
3,2022-11-18,0.0382,0.0451,-0.0069
4,2022-11-17,0.0377,0.0443,-0.0066
...,...,...,...,...
5968,2000-01-07,0.0652,0.0631,0.0021
5969,2000-01-06,0.0657,0.0635,0.0022
5970,2000-01-05,0.0662,0.0638,0.0024
5971,2000-01-04,0.0649,0.0630,0.0019


In [176]:
temp = temp.sort_values('date', ascending=True)

In [177]:
temp

,date,value_10years,value_2years,10Y-2Y spread
5972,2000-01-03,0.0658,0.0638,0.0020
5971,2000-01-04,0.0649,0.0630,0.0019
5970,2000-01-05,0.0662,0.0638,0.0024
5969,2000-01-06,0.0657,0.0635,0.0022
5968,2000-01-07,0.0652,0.0631,0.0021
...,...,...,...,...
4,2022-11-17,0.0377,0.0443,-0.0066
3,2022-11-18,0.0382,0.0451,-0.0069
2,2022-11-21,0.0383,0.0448,-0.0065
1,2022-11-22,0.0376,0.0447,-0.0071


In [171]:
temp['unemployment_change'] = temp['unemployment_rate'].pct_change()

In [172]:
temp

,date,unemployment_rate,unemployment_change
273,2000-01-01,0.040,NaN
272,2000-02-01,0.041,0.025000
271,2000-03-01,0.040,-0.024390
270,2000-04-01,0.038,-0.050000
269,2000-05-01,0.040,0.052632
...,...,...,...
4,2022-06-01,0.036,0.000000
3,2022-07-01,0.035,-0.027778
2,2022-08-01,0.037,0.057143
1,2022-09-01,0.035,-0.054054


In [178]:
temp.to_csv('unemployment.csv')

In [137]:
url = 'path'
temp = pd.read_csv(url, index_col=0)
temp['timestamp'] = pd.to_datetime(temp['timestamp'])
temp = temp.rename(columns = {'timestamp':'date', 'value': 'indicator'})
temp_mask = temp['date'] >= '2000-01-01'
temp = temp[temp_mask]
temp = temp.sort_values('date', ascending=True)
temp['indicator_change'] = temp['indicator'].pct_change()
temp.to_csv('indicator.csv')

In [138]:
temp

,date,consumer_sentiment
273,2000-01-01,112.0
272,2000-02-01,111.3
271,2000-03-01,107.1
270,2000-04-01,109.2
269,2000-05-01,110.7
...,...,...
4,2022-06-01,50
3,2022-07-01,51.5
2,2022-08-01,58.2
1,2022-09-01,58.6


In [141]:
temp['consumer_sentiment'] = pd.to_numeric(temp['consumer_sentiment'])

In [142]:
temp.dtypes

date                  datetime64[ns]
consumer_sentiment           float64
dtype: object

In [143]:
temp['consumer_change'] = temp['consumer_sentiment'].pct_change()

In [144]:
temp

,date,consumer_sentiment,consumer_change
273,2000-01-01,112.0,NaN
272,2000-02-01,111.3,-0.006250
271,2000-03-01,107.1,-0.037736
270,2000-04-01,109.2,0.019608
269,2000-05-01,110.7,0.013736
...,...,...,...
4,2022-06-01,50.0,-0.143836
3,2022-07-01,51.5,0.030000
2,2022-08-01,58.2,0.130097
1,2022-09-01,58.6,0.006873


In [145]:
temp.to_csv('consumer_sentiment.csv')

In [75]:
temp

,date,CPI,CPI_change
273,2000-01-01,168.800,NaN
272,2000-02-01,169.800,NaN
271,2000-03-01,171.200,NaN
270,2000-04-01,171.300,NaN
269,2000-05-01,171.500,NaN
...,...,...,...
4,2022-06-01,296.311,0.090598
3,2022-07-01,296.276,0.085248
2,2022-08-01,296.171,0.082627
1,2022-09-01,296.808,0.082017


In [76]:
temp.to_csv('CPI.csv')